In [ ]:
#autosklearn_env 커널 선택
!pip install autogluon.tabular


In [2]:
import numpy as np
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from autogluon.tabular import TabularPredictor

/Users/jaesolshin/autosklearn_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 데이터 로드 및 샘플링
train_origin = pd.read_csv('/Users/jaesolshin/내 드라이브/2024-2/Google ML Bootcamp2024/data/playground1/train.csv')
train = train_origin.sample(frac=0.01, random_state=42)

# 범주형 변수를 팩터로 변환 (카테고리형)
train.iloc[:, [1, 3, 4, 5, 6, 7, 9]] = train.iloc[:, [1, 3, 4, 5, 6, 7, 9]].astype('category')

# 최소-최대 정규화 (Min-Max 스케일링)
scaler = MinMaxScaler()
train.iloc[:, [2, 8, 10]] = scaler.fit_transform(train.iloc[:, [2, 8, 10]])

# 이분변수 생성: "Annual_Premium" == 2630.0 인 경우
train['Annual_Premium_Binary'] = (train['Annual_Premium'] == 2630.0).astype('category')

# 로그 변환된 "Annual_Premium" 변수 생성
train['Annual_Premium_Log'] = np.where(train['Annual_Premium'] > 0, np.log1p(train['Annual_Premium']), 0)

# 예측에 필요 없는 'id'와 'Annual_Premium' 변수를 드롭
train = train.drop(columns=['id', 'Annual_Premium'])

# 원-핫 인코딩 (One-Hot Encoding)
category_columns = ['Gender', 'Driving_License', 'Region_Code', 'Previously_Insured', 'Vehicle_Age', 'Vehicle_Damage', 'Policy_Sales_Channel', 'Annual_Premium_Binary']
train = pd.get_dummies(train, columns=category_columns, drop_first=True, dtype=int)

# XGBoost에서 발생하는 문제 해결
train.columns = train.columns.str.replace('[', '').str.replace(']', '').str.replace('<', '')

# 특징과 레이블 분리
X = train.drop(columns=['Response'])
y = train['Response']

# 훈련 세트와 테스트 세트로 데이터 분할
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.metrics import roc_auc_score

# AutoGluon을 사용하여 예측 작업 수행
predictor = TabularPredictor(label='Response', eval_metric='accuracy').fit(train_data=pd.concat([X_train, y_train], axis=1))

In [12]:
# 모델 평가
performance = predictor.evaluate(pd.concat([X_valid, y_valid], axis=1))
print(performance)

# 예측 수행 (확률값)
y_proba = predictor.predict_proba(X_valid).iloc[:, 1]

# ROC AUC 계산
roc_auc = roc_auc_score(y_valid, y_proba)
print("\nROC AUC: ", roc_auc)


{'accuracy': 0.8741416775315081, 'balanced_accuracy': 0.5143408071829141, 'mcc': 0.08507864154862346, 'roc_auc': 0.808660613767824, 'f1': 0.0682110682110682, 'precision': 0.3719298245614035, 'recall': 0.0375487070492384}
ROC AUC:  0.8086606137678242
